In [ ]:
# !pip install openai==0.28

In [ ]:
# !pip install pymupdf

In [ ]:
# pip install elevenlabs


In [ ]:
# !pip install gTTS

In [ ]:
import openai
import fitz  # PyMuPDF for PDF handling
import json

# Initialize OpenAI API key
openai.api_key = "" 

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            text += page.get_text()
    return text

# Function to generate a CV summary and identify skills
def generate_cv_summary_and_identify_skills(cv_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant summarizing a CV for interview preparation."},
            {"role": "user", "content": f"Summarize the following CV and identify key skills and expertise areas:\n\n{cv_text}"}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    summary = response.choices[0].message['content'].strip()

    # Extract skills
    skills_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a system identifying the main skills for interview questions."},
            {"role": "user", "content": f"Based on this CV summary:\n\n{summary}\n\nList main skills."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    skills = skills_response.choices[0].message['content'].strip()
    return summary, skills.splitlines()

# Function to generate progressively difficult questions based on skills and job description
def generate_questions_by_skill(skill, job_description):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant creating interview questions for a specific skill aligned with a job description."},
            {"role": "user", "content": f"Using the job description:\n\n{job_description}\n\nGenerate a basic, intermediate, and advanced interview question for the skill '{skill}'. Format the questions as a JSON object with keys 'basic', 'intermediate', and 'advanced'."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    questions_text = response.choices[0].message['content'].strip()

    try:
        questions = json.loads(questions_text)  # Parse JSON into a Python dictionary
    except json.JSONDecodeError:
        print(f"Error: Could not parse JSON for skill '{skill}'. Response text:\n{questions_text}")
        questions = None

    return questions

# Function to generate 12 unique questions for the interview
def generate_unique_questions(cv_summary, skills, job_description):
    questions = []
    seen_questions = set()

    for skill in skills:
        skill_questions = generate_questions_by_skill(skill, job_description)
        if skill_questions:
            for level in ['basic', 'intermediate', 'advanced']:
                question = skill_questions.get(level)
                if question and question not in seen_questions:
                    questions.append({"skill": skill, "level": level, "question": question})
                    seen_questions.add(question)
                if len(questions) == 12:
                    break
        if len(questions) == 12:
            break

    return questions

# Function to evaluate candidate answers based on the CV summary
def evaluate_answer(answer, expected_topic):
    evaluation_prompt = f"Evaluate the following answer based on the topic '{expected_topic}' from the CV summary. Does it align well with the expected expertise? Answer: {answer}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant providing a score and feedback for the candidate's answer."},
            {"role": "user", "content": evaluation_prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    feedback = response.choices[0].message['content'].strip()

    return feedback

# Function to generate a formatted performance report
def generate_formatted_performance_report(result_card):
    performance_data = json.dumps(result_card, indent=2)

    prompt = f"""
Using the following interview result data:

{performance_data}

Generate a summary in the following format:

Overall Result: X out of 12 points (Y%).

Strengths:

[Skill Area (Basic/Intermediate/Advanced):]

• Bullet point 1.
• Bullet point 2.
• Bullet point 3.

Weaknesses:

[Skill Area:]

• Bullet point 1.
• Bullet point 2.
• Bullet point 3.

Conclusion: A concise conclusion summarizing strengths and weaknesses.
"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant generating a formatted performance report based on interview results."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Function to conduct an interactive interview
def conduct_interactive_interview(questions, cv_summary):
    interview_responses = []
    total_score = 0

    print("Bot: Welcome to the interview assistant.")
    print("Bot: Let's start with a few casual questions to get to know you better.")

    name = input("Bot: What is your name? ").strip()
    location = input(f"Bot: Nice to meet you, {name}! Where are you currently located? ").strip()
    hobbies = input("Bot: That's great! What are some of your hobbies? ").strip()

    print("\nBot: Thank you for sharing! Now let's move to the interview questions.\n")

    for i, q in enumerate(questions):
        print(f"Question {i + 1} ({q['level'].capitalize()} - {q['skill']}): {q['question']}")
        candidate_response = input("Your Answer: ").strip()

        if candidate_response.lower() == "quit":
            print("Bot: Interview ended. Thank you for your time!")
            break

        feedback = evaluate_answer(candidate_response, q['skill'])

        interview_responses.append({
            "skill": q['skill'],
            "level": q['level'],
            "question": q['question'],
            "answer": candidate_response,
            "feedback": feedback
        })

        if "good" in feedback or "excellent" in feedback:
            total_score += 1
        elif "average" in feedback:
            total_score += 0.5

    score_percentage = (total_score / 12) * 100

    result_card = {
        "total_score": total_score,
        "score_percentage": score_percentage,
        "feedback": interview_responses
    }

    print("\nInterview Complete. Generating Performance Report...\n")

    # Generate and display the formatted performance report
    report = generate_formatted_performance_report(result_card)
    print(report)

    # Save the report to a file
    with open("Formatted_Performance_Report.txt", "w") as file:
        file.write(report)
    print("\nFormatted Performance Report saved to 'Formatted_Performance_Report.txt'.")

    return result_card

# Main function to conduct the entire interview process
def full_interview_process():
    cv_pdf_path = "Resume Awias.pdf"  # Replace with the actual CV file path
    job_pdf_path = "AI Job Post.pdf"  # Replace with the actual Job Description file path

    try:
        cv_text = extract_text_from_pdf(cv_pdf_path)
        job_description = extract_text_from_pdf(job_pdf_path)

        cv_summary, skills = generate_cv_summary_and_identify_skills(cv_text)
        print("\nDetailed CV Summary:\n", cv_summary)

        questions = generate_unique_questions(cv_summary, skills, job_description)

        if questions:
            conduct_interactive_interview(questions, cv_summary)
        else:
            print("Interview could not proceed due to issues in question generation.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the full interview process
if __name__ == "__main__":
    full_interview_process()


An error occurred: no such file: 'Resume Awias.pdf'


In [1]:
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
from elevenlabs.client import ElevenLabs

def text_to_speech(text):
    client = ElevenLabs(
    api_key='sk_2b3fd08d38c7744e307ea42db37b8b9529f46f94424737e8',
    )
    # Get streamed audio
    audio_stream = client.text_to_speech.convert_as_stream(
        text=text,
        voice_id="JBFqnCBsd6RMkjVDRZzb",
        model_id="eleven_multilingual_v2"
    )

    # Collect the audio bytes
    audio_bytes = b"".join(chunk for chunk in audio_stream if isinstance(chunk, bytes))

    # Convert to an AudioSegment
    audio_segment = AudioSegment.from_file(BytesIO(audio_bytes), format="mp3")

    # Play the audio
    play(audio_segment)


In [2]:
import speech_recognition as sr
import pyaudio

def speech_to_text():   
    mic_index = 3  # Replace with the correct index from the list

    init_rec = sr.Recognizer()
    with sr.Microphone(device_index=mic_index) as source:
        print("Adjusting to ambient noise...")
        init_rec.adjust_for_ambient_noise(source, duration=1)  # Adjust for noise
        print("Start Speaking Listening...")
        audio_data = init_rec.listen(source)
        
    try:
        text = init_rec.recognize_google(audio_data)
        print("You said:", text)
    except sr.UnknownValueError:
        print("Sorry, could not understand the audio.")
    except sr.RequestError:
        print("Could not request results. Check your internet connection.")
    return text

In [ ]:
import openai
import fitz  # PyMuPDF for PDF handling
import json

# Initialize OpenAI API key
openai.api_key = ""  

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            text += page.get_text()
    return text

# Function to collect pre-interview conditions
def collect_pre_interview_conditions():
    print("\nBot: Before we begin the interview, I need to collect some important information.")

    employment_type = input("Bot: Are you applying for a freelance/consulting position or full-time employment? (F/T): ").strip().upper()

    conditions = {}

    if employment_type == 'F':  # Freelance/Consulting
        print("\nBot: Please provide the following information for freelance/consulting work:")
        conditions['employment_type'] = 'Freelance/Consulting'
        conditions['availability_percentage'] = input("Bot: What is your availability percentage? (e.g., 50%, 75%, 100%): ").strip()
        conditions['remote_rate'] = input("Bot: What is your hourly rate for remote work? ($): ").strip()
        conditions['onsite_rate'] = input("Bot: What is your hourly rate for on-site work? ($): ").strip()
        conditions['travel_willingness'] = input("Bot: Are you willing to travel? (Yes/No): ").strip()
        conditions['other_details'] = input("Bot: Any other details (planned vacations, unavailable days, etc.)?: ").strip()

    else:  # Full-time employment
        print("\nBot: Please provide the following information for full-time employment:")
        conditions['employment_type'] = 'Full-time Employment'
        conditions['start_date'] = input("Bot: What is your earliest possible start date? (DD/MM/YYYY): ").strip()
        conditions['work_hours'] = input("Bot: Preferred work hours per week (30/35/40): ").strip()
        conditions['salary_expectations'] = input("Bot: What are your salary expectations? ($): ").strip()
        conditions['work_location'] = input("Bot: Preferred work location? (Remote/On-site/Hybrid): ").strip()

    return conditions

# Helper function to format pre-interview conditions
def format_pre_interview_conditions(conditions):
    if not conditions:
        return "No pre-interview conditions provided."

    formatted = f"Employment Type: {conditions.get('employment_type', 'Not specified')}\n"

    if conditions.get('employment_type') == 'Freelance/Consulting':
        formatted += f"Availability: {conditions.get('availability_percentage', 'Not specified')}\n"
        formatted += f"Remote Rate: ${conditions.get('remote_rate', 'Not specified')}/hour\n"
        formatted += f"On-site Rate: ${conditions.get('onsite_rate', 'Not specified')}/hour\n"
        formatted += f"Willing to Travel: {conditions.get('travel_willingness', 'Not specified')}\n"
        formatted += f"Other Details: {conditions.get('other_details', 'None provided')}"
    else:
        formatted += f"Start Date: {conditions.get('start_date', 'Not specified')}\n"
        formatted += f"Work Hours: {conditions.get('work_hours', 'Not specified')} hours/week\n"
        formatted += f"Salary Expectations: ${conditions.get('salary_expectations', 'Not specified')}\n"
        formatted += f"Work Location: {conditions.get('work_location', 'Not specified')}"

    return formatted

# Function to generate a CV summary and identify skills
def generate_cv_summary_and_identify_skills(cv_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant summarizing a CV for interview preparation."},
            {"role": "user", "content": f"Summarize the following CV and identify key skills and expertise areas:\n\n{cv_text}"}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    summary = response.choices[0].message['content'].strip()

    # Extract skills
    skills_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a system identifying the main skills for interview questions."},
            {"role": "user", "content": f"Based on this CV summary:\n\n{summary}\n\nList main skills."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    skills = skills_response.choices[0].message['content'].strip()
    return summary, skills.splitlines()

# Function to generate progressively difficult questions based on skills and job description
def generate_questions_by_skill(skill, job_description):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an HR assistant creating interview questions for a specific skill aligned with a job description."},
                {"role": "user", "content": f"Using the job description:\n\n{job_description}\n\nGenerate a basic, intermediate, and advanced interview question for the skill '{skill}'. Format the questions as a JSON object with keys 'basic', 'intermediate', and 'advanced'."}
            ],
            max_tokens=150,
            temperature=0.7
        )
        questions_text = response.choices[0].message['content'].strip()

        # Clean the response to ensure it's valid JSON
        questions_text = questions_text.replace('\n', ' ').strip()
        if not questions_text.startswith('{'):
            questions_text = '{' + questions_text
        if not questions_text.endswith('}'):
            questions_text = questions_text + '}'

        return json.loads(questions_text)
    except Exception as e:
        print(f"Error generating questions for skill '{skill}': {str(e)}")
        return {
            "basic": f"Can you explain your experience with {skill}?",
            "intermediate": f"What challenges have you faced when working with {skill}?",
            "advanced": f"How would you implement {skill} in a complex project?"
        }

# Function to generate 12 unique questions for the interview
def generate_unique_questions(cv_summary, skills, job_description):
    questions = []
    seen_question_texts = set()  # Track unique question texts

    for skill in skills:
        if len(questions) >= 12:
            break

        skill_questions = generate_questions_by_skill(skill, job_description)
        if skill_questions:
            for level in ['basic', 'intermediate', 'advanced']:
                question_text = skill_questions.get(level, '')
                if question_text and question_text not in seen_question_texts:
                    questions.append({
                        "skill": skill,
                        "level": level,
                        "question": question_text
                    })
                    seen_question_texts.add(question_text)

                if len(questions) >= 12:
                    break

    return questions[:12]  # Ensure we only return 12 questions

# Function to evaluate candidate answers based on the CV summary
def evaluate_answer(answer, expected_topic):
    evaluation_prompt = f"Evaluate the following answer based on the topic '{expected_topic}' from the CV summary. Does it align well with the expected expertise? Answer: {answer}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant providing a score and feedback for the candidate's answer."},
            {"role": "user", "content": evaluation_prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    feedback = response.choices[0].message['content'].strip()

    return feedback

# Function to generate a formatted performance report
def generate_formatted_performance_report(result_card):
    performance_data = json.dumps(result_card, indent=2)

    prompt = f"""
Using the following interview result data:

{performance_data}

Generate a summary in the following format:

Pre-Interview Conditions:
{format_pre_interview_conditions(result_card.get('pre_interview_conditions', {}))}

Overall Result: X out of 12 points (Y%).

Strengths:

[Skill Area (Basic/Intermediate/Advanced):]

• Bullet point 1.
• Bullet point 2.
• Bullet point 3.

Weaknesses:

[Skill Area:]

• Bullet point 1.
• Bullet point 2.
• Bullet point 3.

Conclusion: A concise conclusion summarizing strengths and weaknesses.
"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an HR assistant generating a formatted performance report based on interview results."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Function to conduct an interactive interview
def conduct_interactive_interview(questions, cv_summary):
    interview_responses = []
    total_score = 0

    print("Bot: Welcome to the interview assistant.")
    print("Bot: Let's start with a few casual questions to get to know you better.")

    name = input("Bot: What is your name? ").strip()
    location = input(f"Bot: Nice to meet you, {name}! Where are you currently located? ").strip()
    hobbies = input("Bot: That's great! What are some of your hobbies? ").strip()

    # Collect pre-interview conditions
    pre_interview_conditions = collect_pre_interview_conditions()

    print("\nBot: Thank you for sharing! Now let's move to the interview questions.\n")

    for i, q in enumerate(questions):
        print(f"Question {i + 1} ({q['level'].capitalize()} - {q['skill']}): {q['question']}")
        candidate_response = input("Your Answer: ").strip()
        

        if candidate_response.lower() == "stop":
            print("Bot: Interview ended. Thank you for your time!")
            break

        feedback = evaluate_answer(candidate_response, q['skill'])

        interview_responses.append({
            "skill": q['skill'],
            "level": q['level'],
            "question": q['question'],
            "answer": candidate_response,
            "feedback": feedback
        })

        if "good" in feedback.lower() or "excellent" in feedback.lower():
            total_score += 1
        elif "average" in feedback.lower():
            total_score += 0.5

    score_percentage = (total_score / 12) * 100

    result_card = {
        "total_score": total_score,
        "score_percentage": score_percentage,
        "feedback": interview_responses,
        "pre_interview_conditions": pre_interview_conditions
    }

    print("\nInterview Complete. Generating Performance Report...\n")

    # Generate and display the formatted performance report
    report = generate_formatted_performance_report(result_card)
    print(report)

    # Save the report to a file
    with open("Formatted_Performance_Report.txt", "w") as file:
        file.write(report)
    print("\nFormatted Performance Report saved to 'Formatted_Performance_Report.txt'.")

    return result_card

# Main function to conduct the entire interview process
def full_interview_process():
    cv_pdf_path = "D:/Coding/Job/AI Interviewer/Sheheryar_Resume - Sheheryar Ramzan.pdf"  # Replace with the actual CV file path
    job_pdf_path = "D:/Coding/Job/AI Interviewer/Machine_Learning_Job_Description.pdf"  # Replace with the actual Job Description file path

    try:
        cv_text = extract_text_from_pdf(cv_pdf_path)
        job_description = extract_text_from_pdf(job_pdf_path)

        cv_summary, skills = generate_cv_summary_and_identify_skills(cv_text)
        print("\nDetailed CV Summary:\n", cv_summary)

        questions = generate_unique_questions(cv_summary, skills, job_description)

        if questions:
            conduct_interactive_interview(questions, cv_summary)
        else:
            print("Interview could not proceed due to issues in question generation.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the full interview process
if __name__ == "__main__":
    full_interview_process()


Detailed CV Summary:
 Sheheryar Ramzan is a Computer Science student with a GPA of 3.51/4.00 from FAST National University. He has expertise in areas such as Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, and Generative AI. Key skills include Python programming, TensorFlow, Keras, PyTorch, Scikit-learn, OpenCV, and Numpy. Sheheryar has experience in algorithm engineering, machine learning, and teaching assistance. He has worked on projects related to cricket shot analysis, contraband detection, Parkinson's disease classification, and more. Sheheryar also has certifications in Generative AI and Neural Networks and has received awards for academic excellence.
Bot: Welcome to the interview assistant.
Bot: Let's start with a few casual questions to get to know you better.

Bot: Before we begin the interview, I need to collect some important information.

Bot: Please provide the following information for freelance/consulting work:

Bot: Thank you for sharing! No

In [ ]:
import speech_recognition as sr
import pyaudio

mic_index = 3  # Replace with the correct index from the list

init_rec = sr.Recognizer()
with sr.Microphone(device_index=mic_index) as source:
    print("Adjusting to ambient noise...")
    init_rec.adjust_for_ambient_noise(source, duration=1)  # Adjust for noise
    print("Start Speaking Listening...")
    audio_data = init_rec.listen(source)
    
try:
    text = init_rec.recognize_google(audio_data)
    print("You said:", text)
except sr.UnknownValueError:
    print("Sorry, could not understand the audio.")
except sr.RequestError:
    print("Could not request results. Check your internet connection.")



Adjusting to ambient noise...
Start Speaking Listening...
You said: hello mike testing for Google text to speech


In [ ]:
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
from elevenlabs.client import ElevenLabs

client = ElevenLabs(
  api_key='sk_2b3fd08d38c7744e307ea42db37b8b9529f46f94424737e8',
)
# Get streamed audio
audio_stream = client.text_to_speech.convert_as_stream(
    text=text,
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2"
)

# Collect the audio bytes
audio_bytes = b"".join(chunk for chunk in audio_stream if isinstance(chunk, bytes))

# Convert to an AudioSegment
audio_segment = AudioSegment.from_file(BytesIO(audio_bytes), format="mp3")

# Play the audio
play(audio_segment)
